# A full business solution

## Now we will take our project from Day 1 to the next level

### BUSINESS CHALLENGE:

Create a product that builds a Brochure for a company to be used for prospective clients, investors and potential recruits.

We will be provided a company name and their primary website.

See the end of this notebook for examples of real-world business applications.

And remember: I'm always available if you have problems or ideas! Please do reach out.

In [3]:
# imports

import os
import requests
import json
from typing import List
from dotenv import load_dotenv
from bs4 import BeautifulSoup
from IPython.display import Markdown, display, update_display
from openai import OpenAI

In [4]:
# Initialize and constants

load_dotenv()
os.environ['OPENAI_API_KEY'] = os.getenv('OPENAI_API_KEY', 'your-key-if-not-using-env')
MODEL = 'gpt-4o-mini'
openai = OpenAI()

In [5]:
# A class to represent a Webpage

class Website:
    """
    A utility class to represent a Website that we have scraped, now with links
    """
    url: str
    title: str
    body: str
    links: List[str]
    text: str

    def __init__(self, url):
        self.url = url
        response = requests.get(url)
        self.body = response.content
        soup = BeautifulSoup(self.body, 'html.parser')
        self.title = soup.title.string if soup.title else "No title found"
        if soup.body:
            for irrelevant in soup.body(["script", "style", "img", "input"]):
                irrelevant.decompose()
            self.text = soup.body.get_text(separator="\n", strip=True)
        else:
            self.text = ""
        links = [link.get('href') for link in soup.find_all('a')]
        self.links = [link for link in links if link]

    def get_contents(self):
        return f"Webpage Title:\n{self.title}\nWebpage Contents:\n{self.text}\n\n"

In [6]:
ed = Website("https://edwarddonner.com")
print(ed.get_contents())

Webpage Title:
Home - Edward Donner
Webpage Contents:
Home
Outsmart
An arena that pits LLMs against each other in a battle of diplomacy and deviousness
About
Posts
Well, hi there.
I’m Ed. I like writing code and experimenting with LLMs, and hopefully you’re here because you do too. I also enjoy DJing (but I’m badly out of practice), amateur electronic music production (
very
amateur) and losing myself in
Hacker News
, nodding my head sagely to things I only half understand.
I’m the co-founder and CTO of
Nebula.io
. We’re applying AI to a field where it can make a massive, positive impact: helping people discover their potential and pursue their reason for being. Recruiters use our product today to source, understand, engage and manage talent. I’m previously the founder and CEO of AI startup untapt,
acquired in 2021
.
We work with groundbreaking, proprietary LLMs verticalized for talent, we’ve
patented
our matching model, and our award-winning platform has happy customers and tons of pr

In [7]:
print(ed.links)

['https://edwarddonner.com/', 'https://edwarddonner.com/outsmart/', 'https://edwarddonner.com/about-me-and-about-nebula/', 'https://edwarddonner.com/posts/', 'https://edwarddonner.com/', 'https://news.ycombinator.com', 'https://nebula.io/?utm_source=ed&utm_medium=referral', 'https://www.prnewswire.com/news-releases/wynden-stark-group-acquires-nyc-venture-backed-tech-startup-untapt-301269512.html', 'https://patents.google.com/patent/US20210049536A1/', 'https://www.linkedin.com/in/eddonner/', 'https://edwarddonner.com/2024/10/16/from-software-engineer-to-ai-data-scientist-resources/', 'https://edwarddonner.com/2024/10/16/from-software-engineer-to-ai-data-scientist-resources/', 'https://edwarddonner.com/2024/08/06/outsmart/', 'https://edwarddonner.com/2024/08/06/outsmart/', 'https://edwarddonner.com/2024/06/26/choosing-the-right-llm-resources/', 'https://edwarddonner.com/2024/06/26/choosing-the-right-llm-resources/', 'https://edwarddonner.com/2024/02/07/fine-tune-llm-on-texts-a-simulation

## First step: Have GPT-4o-mini figure out which links are relevant

### Use a call to gpt-4o-mini to read the links on a webpage, and respond in structured JSON.  
It should decide which links are relevant, and replace relative links such as "/about" with "https://company.com/about".  
We will use "one shot prompting" in which we provide an example of how it should respond in the prompt.

This is an excellent use case for an LLM, because it requires nuanced understanding. Imagine trying to code this without LLMs by parsing and analyzing the webpage - it would be very hard!

Sidenote: there is a more advanced technique called "Structured Outputs" in which we require the model to respond according to a spec. We cover this technique in Week 8 during our autonomous Agentic AI project.

In [9]:
link_system_prompt = "You are provided with a list of links found on a webpage. \
You are able to decide which of the links would be most relevant to include in a brochure about the company, \
such as links to an About page, or a Company page, or Careers/Jobs pages.\n"
link_system_prompt += "You should respond in JSON as in this example:"
link_system_prompt += """
{
    "links": [
        {"type": "about page", "url": "https://full.url/goes/here/about"},
        {"type": "careers page": "url": "https://another.full.url/careers"}
    ]
}
"""

In [10]:
def get_links_user_prompt(website):
    user_prompt = f"Here is the list of links on the website of {website.url} - "
    user_prompt += "please decide which of these are relevant web links for a brochure about the company, respond with the full https URL in JSON format. \
Do not include Terms of Service, Privacy, email links.\n"
    user_prompt += "Links (some might be relative links):\n"
    user_prompt += "\n".join(website.links)
    return user_prompt

In [11]:
print(get_links_user_prompt(ed))

Here is the list of links on the website of https://edwarddonner.com - please decide which of these are relevant web links for a brochure about the company, respond with the full https URL in JSON format. Do not include Terms of Service, Privacy, email links.
Links (some might be relative links):
https://edwarddonner.com/
https://edwarddonner.com/outsmart/
https://edwarddonner.com/about-me-and-about-nebula/
https://edwarddonner.com/posts/
https://edwarddonner.com/
https://news.ycombinator.com
https://nebula.io/?utm_source=ed&utm_medium=referral
https://www.prnewswire.com/news-releases/wynden-stark-group-acquires-nyc-venture-backed-tech-startup-untapt-301269512.html
https://patents.google.com/patent/US20210049536A1/
https://www.linkedin.com/in/eddonner/
https://edwarddonner.com/2024/10/16/from-software-engineer-to-ai-data-scientist-resources/
https://edwarddonner.com/2024/10/16/from-software-engineer-to-ai-data-scientist-resources/
https://edwarddonner.com/2024/08/06/outsmart/
https://e

In [12]:
def get_links(url):
    website = Website(url)
    completion = openai.chat.completions.create(
        model=MODEL,
        messages=[
            {"role": "system", "content": link_system_prompt},
            {"role": "user", "content": get_links_user_prompt(website)}
      ],
        response_format={"type": "json_object"}
    )
    result = completion.choices[0].message.content
    return json.loads(result)

In [13]:
get_links("https://anthropic.com")

{'links': [{'type': 'about page', 'url': 'https://anthropic.com/company'},
  {'type': 'careers page', 'url': 'https://anthropic.com/careers'},
  {'type': 'team page', 'url': 'https://anthropic.com/team'},
  {'type': 'research page', 'url': 'https://anthropic.com/research'},
  {'type': 'enterprise page', 'url': 'https://anthropic.com/enterprise'}]}

## Second step: make the brochure!

Assemble all the details into another prompt to GPT4-o

In [14]:
def get_all_details(url):
    result = "Landing page:\n"
    result += Website(url).get_contents()
    links = get_links(url)
    print("Found links:", links)
    for link in links["links"]:
        result += f"\n\n{link['type']}\n"
        result += Website(link["url"]).get_contents()
    return result

In [15]:
print(get_all_details("https://anthropic.com"))

Found links: {'links': [{'type': 'about page', 'url': 'https://anthropic.com/company'}, {'type': 'careers page', 'url': 'https://anthropic.com/careers'}, {'type': 'team page', 'url': 'https://anthropic.com/team'}, {'type': 'research page', 'url': 'https://anthropic.com/research'}]}
Landing page:
Webpage Title:
Home \ Anthropic
Webpage Contents:
Claude
Overview
Team
Enterprise
API
Pricing
Research
Company
Careers
News
AI
research
and
products
that put safety at the frontier
Claude.ai
Meet Claude 3.5 Sonnet
Claude 3.5 Sonnet, our most intelligent AI model, is now available.
Talk to Claude
API
Build with Claude
Start using Claude to drive efficiency and create new revenue streams.
Learn more
Announcements
Introducing computer use, a new Claude 3.5 Sonnet, and Claude 3.5 Haiku
Oct 22, 2024
Model updates
3.5 Sonnet
3.5 Haiku
Our Work
Product
Claude for Enterprise
Sep 4, 2024
Alignment
·
Research
Constitutional AI: Harmlessness from AI Feedback
Dec 15, 2022
Announcements
Core Views on AI Saf

In [16]:
system_prompt = "You are an assistant that analyzes the contents of several relevant pages from a company website \
and creates a short brochure about the company for prospective customers, investors and recruits. Respond in markdown.\
Include details of company culture, customers and careers/jobs if you have the information."

# Or uncomment the lines below for a more humorous brochure - this demonstrates how easy it is to incorporate 'tone':

# system_prompt = "You are an assistant that analyzes the contents of several relevant pages from a company website \
# and creates a short humorous, entertaining, jokey brochure about the company for prospective customers, investors and recruits. Respond in markdown.\
# Include details of company culture, customers and careers/jobs if you have the information."


In [17]:
def get_brochure_user_prompt(company_name, url):
    user_prompt = f"You are looking at a company called: {company_name}\n"
    user_prompt += f"Here are the contents of its landing page and other relevant pages; use this information to build a short brochure of the company in markdown.\n"
    user_prompt += get_all_details(url)
    user_prompt = user_prompt[:20_000] # Truncate if more than 20,000 characters
    return user_prompt

In [18]:
def create_brochure(company_name, url):
    response = openai.chat.completions.create(
        model=MODEL,
        messages=[
            {"role": "system", "content": system_prompt},
            {"role": "user", "content": get_brochure_user_prompt(company_name, url)}
          ],
    )
    result = response.choices[0].message.content
    display(Markdown(result))

In [19]:
create_brochure("Anthropic", "https://anthropic.com")

Found links: {'links': [{'type': 'about page', 'url': 'https://anthropic.com/company'}, {'type': 'careers page', 'url': 'https://anthropic.com/careers'}, {'type': 'team page', 'url': 'https://anthropic.com/team'}, {'type': 'enterprise page', 'url': 'https://anthropic.com/enterprise'}, {'type': 'api page', 'url': 'https://anthropic.com/api'}, {'type': 'pricing page', 'url': 'https://anthropic.com/pricing'}, {'type': 'research page', 'url': 'https://anthropic.com/research'}, {'type': 'news page', 'url': 'https://anthropic.com/news'}, {'type': 'jobs page', 'url': 'https://anthropic.com/jobs'}]}


```markdown
# Anthropic: Pioneering Safety in AI

## Who We Are
Anthropic is an AI safety and research company based in San Francisco, dedicated to building reliable, interpretable, and steerable AI systems. Our interdisciplinary team, composed of researchers, engineers, policy experts, and business leaders, works collaboratively to ensure that AI systems are both beneficial and safe for society.

### Our Mission
At Anthropic, we believe that AI can significantly impact the world. We are dedicated to developing systems that people can depend on while generating crucial research about the opportunities and risks associated with AI.

## Our Products
**Meet Claude**: 
Claude is our cutting-edge AI model available for enterprise and API use, designed to enhance productivity and creativity. Claude functions as a virtual teammate, streamlining tasks, enabling collaboration, and leveraging company knowledge to facilitate intelligent decision-making.

**Features of Claude**:
- Rapid content generation for diverse business needs.
- Seamless integration with existing company knowledge and data.
- Support for various teams including engineering, marketing, sales, and customer support.

## Our Work in Research
Anthropic conducts frontier research in AI safety, exploring domains from interpretability to reinforcement learning from human feedback. We treat AI safety as a systematic science, regularly sharing insights to promote industry-wide safety standards.

## Company Culture
### **Our Values**:
1. **Mission-Driven**: We exist to ensure transformative AI benefits humanity.
2. **High Trust Environment**: We prioritize honesty, emotional maturity, and intellectual openness.
3. **Collaboration**: We view ourselves as one cohesive team, collaborating across various disciplines.
4. **Pragmatism**: We favor sensible, practical solutions and celebrate simplicity.

### **Work Environment**:
We foster a work culture that encourages collaboration, trust, and diversity of thought. Our team members come from distinct backgrounds, including academia, startups, and industry, which enriches our collective intelligence and creativity.

## Careers at Anthropic
We offer a range of opportunities for individuals passionate about AI and its safe development. Our comprehensive benefits include:
- **Health & Wellness**: Comprehensive health, dental, and vision insurance.
- **Parental Leave**: 22 weeks of paid parental leave.
- **Flexible PTO**: Unlimited open vacation policy.
- **Competitive Compensation**: Attractive salaries with equity packages and generous matching for equity donations.

### **Hiring Process**:
Our interview process is thorough and designed to find the best fit for both the company and candidates. It emphasizes skills, experience, and cultural fit through a mix of interviews and assignments, tailored based on the role.

## Join Us
We are looking for motivated individuals who want to contribute to shaping the future of safe AI. If you're ready to make an impact, explore our open roles and become a part of a mission-driven team that is redefining the AI landscape.

### **Connect with Us**:
- **Website**: [Anthropic](https://anthropic.com)
- **LinkedIn**: [Anthropic LinkedIn](https://linkedin.com/company/anthropic)
- **Twitter**: [@Anthropic](https://twitter.com/Anthropic)

---

**Together, let's build a safe future with AI!**
```

## Finally - a minor improvement

With a small adjustment, we can change this so that the results stream back from OpenAI,
with the familiar typewriter animation

In [20]:
def stream_brochure(company_name, url):
    stream = openai.chat.completions.create(
        model=MODEL,
        messages=[
            {"role": "system", "content": system_prompt},
            {"role": "user", "content": get_brochure_user_prompt(company_name, url)}
          ],
        stream=True
    )
    
    response = ""
    display_handle = display(Markdown(""), display_id=True)
    for chunk in stream:
        response += chunk.choices[0].delta.content or ''
        response = response.replace("```","").replace("markdown", "")
        update_display(Markdown(response), display_id=display_handle.display_id)

In [21]:
stream_brochure("Anthropic", "https://anthropic.com")

Found links: {'links': [{'type': 'about page', 'url': 'https://anthropic.com/company'}, {'type': 'careers page', 'url': 'https://anthropic.com/careers'}, {'type': 'team page', 'url': 'https://anthropic.com/team'}, {'type': 'research page', 'url': 'https://anthropic.com/research'}, {'type': 'enterprise page', 'url': 'https://anthropic.com/enterprise'}, {'type': 'api page', 'url': 'https://anthropic.com/api'}, {'type': 'pricing page', 'url': 'https://anthropic.com/pricing'}, {'type': 'news page', 'url': 'https://anthropic.com/news'}]}



# Anthropic Company Brochure

## Overview
**Anthropic** is a leading AI safety and research company headquartered in San Francisco. We focus on building reliable, interpretable, and steerable AI systems that prioritize safety, transforming how businesses operate in a rapidly changing world.

## Our Mission
At Anthropic, we believe in the transformative potential of AI. Our goal is to ensure that advanced AI technologies benefit humanity, addressing the opportunities and risks they present. We employ an interdisciplinary approach, combining expertise from machine learning, physics, policy, and product development to create AI systems that are both effective and safe.

## Our Products
### Claude
We are proud to introduce **Claude**, our intelligent AI model designed to empower businesses. Currently, we offer **Claude 3.5 Sonnet** and other variants through an API, enabling companies to drive efficiency and open new revenue streams. Claude acts as a virtual teammate, enhancing collaboration and productivity across teams.

### Enterprise Solutions
With **Claude for Enterprise**, we provide organizations with AI tools that unlock the potential of their existing knowledge. Our solutions enable secure access to internal expertise, streamlining workflows, and enhancing decision-making across departments.

## Research and Innovation
At Anthropic, we are committed to conducting frontier research in AI safety. Our teams explore a variety of safety techniques, from interpretability to the societal impacts of AI, ensuring that our products not only meet high standards of reliability but can also adapt to evolving challenges.

## Company Culture
### Values
1. **Mission-Driven**: We align our work with our commitment to ensuring transformative AI helps people and society flourish.
2. **High Trust**: We foster an environment of mutual respect, encouraging open dialogue and honesty.
3. **Collaborative Team**: We work as a unified team towards common goals, combining diverse skills and perspectives.
4. **Pragmatism and Empiricism**: We emphasize practical solutions grounded in real-world outcomes.

### Benefits and Work-Life Balance
- Comprehensive health, dental, and vision coverage
- Unlimited paid time off, with most employees taking 4-6 weeks annually
- Support for family needs, including 22 weeks of parental leave
- Flexible wellness stipends, commuter coverage, and home office improvements

## Join Our Team
We are always looking for passionate individuals to contribute to our mission of making AI safe. Our hiring process is designed to be inclusive and thorough, allowing candidates from diverse backgrounds to shine in technical and operational roles.

### Current Openings
Explore exciting career opportunities across engineering, research, policy, and operations. We welcome applicants without specific educational requirements, focusing on skills and potential.

## Our Customers
Anthropic serves a range of businesses, nonprofits, and civil society organizations who seek to leverage AI responsibly. Our tools help them innovate, improve efficiencies, and realize the full potential of their operations.

## Get in Touch
Interested in learning more or joining our journey? Visit our website or connect with us on [Twitter](https://twitter.com/) and [LinkedIn](https://www.linkedin.com/) to explore opportunities to collaborate or join our team.

---

**Anthropic - Building a Future with Safe AI**  
Transforming AI for the benefit of humanity, one step at a time.

This brochure provides a comprehensive yet concise overview of Anthropic's mission, products, company culture, and opportunities, making it suitable for potential customers, investors, and recruits.

In [22]:
stream_brochure("HuggingFace", "https://huggingface.co")

Found links: {'links': [{'type': 'about page', 'url': 'https://huggingface.co/huggingface'}, {'type': 'careers page', 'url': 'https://apply.workable.com/huggingface/'}, {'type': 'company blog', 'url': 'https://huggingface.co/blog'}, {'type': 'github page', 'url': 'https://github.com/huggingface'}, {'type': 'twitter page', 'url': 'https://twitter.com/huggingface'}, {'type': 'linkedin page', 'url': 'https://www.linkedin.com/company/huggingface/'}]}


# Hugging Face Brochure

**Hugging Face**  
*The AI community building the future.*

---

## About Us

Hugging Face is a collaborative platform dedicated to the machine learning community, where individuals come together to create and share models, datasets, and applications. Our mission is to democratize machine learning, making advanced tools accessible to everyone, and to foster innovation and collaboration through open-source contributions.

---

## Our Offerings

- **Models**: Browse over 400,000 machine learning models, including state-of-the-art frameworks like *Transformers* and *Diffusers*.
- **Datasets**: Access more than 100,000 datasets across various modalities, such as text, images, video, and audio.
- **Spaces**: Collaborate on and showcase applications while leveraging our optimized computing infrastructure.
- **Enterprise Solutions**: Custom-tailored platforms for organizations with extensive security measures and dedicated support.

---

## Customer Base

Hugging Face is trusted by over 50,000 organizations globally, including industry giants like:
- **Meta**
- **Amazon Web Services**
- **Google**
- **Microsoft**
- **Grammarly**

---

## Company Culture

At Hugging Face, we pride ourselves on our open-source philosophy and vibrant community. Our team consists of passionate individuals who are dedicated to pushing the boundaries of machine learning. We encourage continuous learning, collaboration, and innovation in a supportive environment. 

*Join us in making an impact! We are always looking for talented individuals to contribute to our mission.*

---

## Careers

We offer a range of job opportunities suitable for diverse talents in the field of AI and machine learning. Joining Hugging Face means being part of a dynamic team that values contribution and creativity. Check our [careers page](https://huggingface.co/jobs) for current openings!

---

## Connect With Us

- **Website**: [huggingface.co](https://huggingface.co)
- **GitHub**: [Hugging Face GitHub](https://github.com/huggingface)
- **Twitter**: [@huggingface](https://twitter.com/huggingface)
- **LinkedIn**: [Hugging Face LinkedIn](https://www.linkedin.com/company/huggingface)

Join the AI revolution and be part of the Hugging Face community today!

## Business Applications

In this exercise we extended the Day 1 code to make multiple LLM calls, and generate a document.

In terms of techniques, this is perhaps the first example of Agentic AI design patterns, as we combined multiple calls to LLMs. This will feature more in Week 2, and then we will return to Agentic AI in a big way in Week 8 when we build a fully autonomous Agent solution.

In terms of business applications - generating content in this way is one of the very most common Use Cases. As with summarization, this can be applied to any business vertical. Write marketing content, generate a product tutorial from a spec, create personalized email content, and so much more. Explore how you can apply content generation to your business, and try making yourself a proof-of-concept prototype.